In [1]:
# import nltk

# !pip install autocorrect
# !pip install -U spacy
# !python -m spacy download en_core_web_sm
# nltk.download('punkt')
# nltk.download('stopwords')
# nltk.download('wordnet')
# nltk.download('omw-1.4')
# spacy.cli.download("en_core_web_sm")
# import warnings
# warnings.filterwarnings('ignore')

In [2]:
import numpy as np
import pandas as pd
from nltk.corpus import stopwords
from autocorrect import Speller
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.svm import SVC
import spacy
import re
nlp = spacy.load('en_core_web_sm')

from sklearn.linear_model import LogisticRegression
from sentence_transformers import SentenceTransformer, losses, models, util
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator
from sentence_transformers.readers import InputExample

2022-12-02 18:51:40.383873: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-02 18:51:40.486518: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-12-02 18:51:40.508362: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-12-02 18:51:40.972620: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; 

In [3]:
data = pd.read_csv("F2_Claim_Check_Worthiness_train.csv")

In [4]:
# preprocessing funciton created
def preprocessing(df):
    df1 = df.copy()
    df1['white_space_removed'] = 0
    df1['tokenized_data'] = 0
    df1['stopword_removed_data'] = 0
    df1['punct_removed_data'] = 0
    df1['url_removed_data'] = 0
    df1['spelling_checked_data'] = 0
    stop_words = set(stopwords.words("english"))

    # create object for lemmetizer and spelling checking
    # lemmatizer = WordNetLemmatizer()
    spell = Speller(lang='en')


    # iterate over each row of dataset and preprocess data
    for i in range(df1.shape[0]):

        # white space removel
        df1['white_space_removed'][i] = re.sub("\s+", " ", df1.tweet[i])


        # lower casing and tokenization
        lower = df1['white_space_removed'][i].lower()
        tokenized_data = word_tokenize(lower)
        df1['tokenized_data'][i] = tokenized_data
        # print(tokenized_data)


        # remove stopwords
        stopword_removed_data = [x for x in tokenized_data if x not in stop_words]
        df1['stopword_removed_data'][i] = stopword_removed_data 
        # print(stopword_removed_data)


        # punctuation removel
        punct_removed_data = [x for x in stopword_removed_data if x.isalnum()]
        df1['punct_removed_data'][i] = punct_removed_data
        # print(punct_removed_data)


        # remove urls and html tags
        urls = re.findall("https?://[a-zA-Z0-9_\?=\@\/#=.~-]+", " ".join(punct_removed_data))
        url_removed_data = [x for x in punct_removed_data if x not in urls]
        df1['url_removed_data'][i] = url_removed_data
        # print(url_removed_data)
        # print(type(url_removed_data))


        # spelling checking
        spelling_checked_data = [spell(x) for x in url_removed_data]
        df1['spelling_checked_data'][i] = " ".join(spelling_checked_data)
        # print(" ".join(spelling_checked_data))

    return df1

In [5]:
df = preprocessing(data)

/tmp/ipykernel_138847/3348583266.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['white_space_removed'][i] = re.sub("\s+", " ", df1.tweet[i])
/tmp/ipykernel_138847/3348583266.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['tokenized_data'][i] = tokenized_data
/tmp/ipykernel_138847/3348583266.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['stopword_removed_data'][i] = stopword_removed_data
/tmp/ipykernel_138

In [6]:
df = df.rename({'spelling_checked_data': 'preprocessed_text'}, axis=1)
df.head()

,tweet,label,id,white_space_removed,tokenized_data,stopword_removed_data,punct_removed_data,url_removed_data,preprocessed_text
0,"India's gift of 100,000 COVID-19 vaccines arri...",0,0,"India's gift of 100,000 COVID-19 vaccines arri...","[india, 's, gift, of, 100,000, covid-19, vacci...","[india, 's, gift, 100,000, covid-19, vaccines,...","[india, gift, vaccines, arrived, barbados, ear...","[india, gift, vaccines, arrived, barbados, ear...",india gift vaccines arrived barbados earlier t...
1,As part of the ongoing nationwide vaccination ...,0,1,As part of the ongoing nationwide vaccination ...,"[as, part, of, the, ongoing, nationwide, vacci...","[part, ongoing, nationwide, vaccination, rollo...","[part, ongoing, nationwide, vaccination, rollo...","[part, ongoing, nationwide, vaccination, rollo...",part ongoing nationwide vaccination rollout se...
2,"Pleased to receive 50,000 doses of Covid-19 va...",0,2,"Pleased to receive 50,000 doses of Covid-19 va...","[pleased, to, receive, 50,000, doses, of, covi...","[pleased, receive, 50,000, doses, covid-19, va...","[pleased, receive, doses, vaccines, india, mor...","[pleased, receive, doses, vaccines, india, mor...",pleased receive doses vaccines india morning g...
3,Four former presidents have banded together fo...,0,3,Four former presidents have banded together fo...,"[four, former, presidents, have, banded, toget...","[four, former, presidents, banded, together, t...","[four, former, presidents, banded, together, t...","[four, former, presidents, banded, together, t...",four former presidents banded together two nat...
4,WSJ: All three of Russia's main intelligence s...,1,4,WSJ: All three of Russia's main intelligence s...,"[wsj, :, all, three, of, russia, 's, main, int...","[wsj, :, three, russia, 's, main, intelligence...","[wsj, three, russia, main, intelligence, servi...","[wsj, three, russia, main, intelligence, servi...",wsj three russia main intelligence services sr...


In [7]:
df_preprocessed = df[['tweet','label','id','preprocessed_text']]

In [8]:
df_preprocessed.to_csv("preprocessed_train.csv", index=False)

In [9]:
df_preprocessed = pd.read_csv("preprocessed_train.csv", index_col=False)
df_preprocessed.head()

,tweet,label,id,preprocessed_text
0,"India's gift of 100,000 COVID-19 vaccines arri...",0,0,india gift vaccines arrived barbados earlier t...
1,As part of the ongoing nationwide vaccination ...,0,1,part ongoing nationwide vaccination rollout se...
2,"Pleased to receive 50,000 doses of Covid-19 va...",0,2,pleased receive doses vaccines india morning g...
3,Four former presidents have banded together fo...,0,3,four former presidents banded together two nat...
4,WSJ: All three of Russia's main intelligence s...,1,4,wsj three russia main intelligence services sr...


In [10]:
def pos_tag_extraction(sentence):
    docs = nlp(sentence)
    pos_tags_list=[]
    for token in docs:
        pos_tags_list.append(token.pos_)
    pos_tags = " ".join(pos_tags_list)
    return pos_tags

In [11]:
x = pos_tag_extraction("table is slleping")
print(x)

NOUN AUX VERB


In [12]:
def  dep_feature_extraction(sentence):
    docs = nlp(sentence)
    dep_list=[]
    for token in docs:
        dep_list.append(token.dep_)
    dep_feature = " ".join(dep_list)
    return dep_feature

In [13]:
dep_feature_extraction(x)

'compound compound ROOT'

In [14]:
def get_pos_vector(pos_sentence):
    pos_tags = {'ADJ':0,'ADV':0,'NOUN':0,'PROPN':0,'VERB':0,'NUM':0,'ADP':0,'PRON':0} 
    
    for pos in pos_sentence.split():        
        if pos in pos_tags:
            pos_tags[pos] += 1
            
    return list(pos_tags.values())

In [15]:
get_pos_vector(x)

[0, 0, 1, 0, 1, 0, 0, 0]

In [16]:
def get_pos_dep_dict(df):#create set of all unique dependencies in entire data    
    unique_dep = set()
    for row in df['preprocessed_text']:
        pos_sentence = pos_tag_extraction(row).split()
        dep_sentence = dep_feature_extraction(row).split()
        for pos, dep in zip(pos_sentence, dep_sentence):
            unique_dep.add(pos+"_"+dep)
    print(unique_dep)
    return list(unique_dep)

unique_dependency_list = get_pos_dep_dict(df_preprocessed)

{'NUM_meta', 'PROPN_prep', 'PROPN_advmod', 'PROPN_intj', 'ADJ_punct', 'PROPN_nmod', 'AUX_conj', 'X_dep', 'INTJ_nmod', 'ADJ_appos', 'AUX_quantmod', 'AUX_compound', 'PRON_poss', 'PROPN_oprd', 'PRON_dobj', 'NOUN_intj', 'ADJ_oprd', 'NOUN_poss', 'PROPN_appos', 'DET_quantmod', 'NOUN_xcomp', 'ADJ_compound', 'VERB_ccomp', 'PROPN_punct', 'PROPN_compound', 'PROPN_npadvmod', 'INTJ_nsubj', 'NUM_pobj', 'ADJ_dobj', 'NOUN_csubj', 'PROPN_meta', 'PROPN_relcl', 'NUM_nmod', 'SCONJ_mark', 'NUM_npadvmod', 'AUX_ROOT', 'NUM_det', 'ADV_cc', 'VERB_ROOT', 'PRON_appos', 'PROPN_amod', 'NUM_compound', 'NOUN_punct', 'NUM_nsubj', 'ADP_cc', 'X_cc', 'VERB_punct', 'VERB_amod', 'ADJ_npadvmod', 'ADJ_ccomp', 'X_amod', 'ADP_ROOT', 'ADV_quantmod', 'PROPN_csubj', 'NOUN_advmod', 'CCONJ_cc', 'PRON_nmod', 'ADP_prep', 'NUM_dative', 'INTJ_prep', 'DET_det', 'ADJ_relcl', 'NOUN_acl', 'VERB_appos', 'ADP_intj', 'ADP_nsubj', 'PROPN_xcomp', 'AUX_dobj', 'SPACE_compound', 'PRON_nsubj', 'ADJ_csubj', 'INTJ_compound', 'NUM_ROOT', 'CCONJ_det'

In [17]:
len(unique_dependency_list)

226

In [18]:
def get_count_pos_dep_dict(df):#create count of all unique dependencies in entire data    
    unique_dep = {}
    for row in df['preprocessed_text']:
        pos_sentence = pos_tag_extraction(row).split()
        dep_sentence = dep_feature_extraction(row).split()
        for pos, dep in zip(pos_sentence, dep_sentence):
            if (pos+"_"+dep) not in unique_dep:
                unique_dep[(pos+"_"+dep)] = 1
            else:
                unique_dep[(pos+"_"+dep)] += 1
            
    return unique_dep

count_dependency_dict = get_count_pos_dep_dict(df_preprocessed)

In [19]:
count_dependency_dict

{'PROPN_compound': 4225,
 'NOUN_compound': 7528,
 'NOUN_nsubj': 3528,
 'VERB_ROOT': 2007,
 'NOUN_dobj': 4428,
 'ADV_advmod': 1330,
 'NOUN_npadvmod': 804,
 'ADJ_amod': 4790,
 'VERB_relcl': 145,
 'VERB_amod': 611,
 'PROPN_nsubj': 1024,
 'VERB_ccomp': 1974,
 'VERB_acl': 874,
 'PRON_nsubj': 101,
 'NOUN_dep': 23,
 'NOUN_ROOT': 114,
 'PROPN_nmod': 565,
 'NOUN_appos': 295,
 'NOUN_nmod': 842,
 'ADJ_compound': 96,
 'NUM_nummod': 1333,
 'PROPN_npadvmod': 170,
 'VERB_xcomp': 472,
 'VERB_conj': 505,
 'PROPN_dobj': 384,
 'ADP_prep': 433,
 'NOUN_pobj': 446,
 'VERB_compound': 208,
 'NUM_dobj': 37,
 'NOUN_pcomp': 1,
 'VERB_prep': 158,
 'VERB_nsubj': 122,
 'VERB_advcl': 589,
 'NOUN_attr': 45,
 'VERB_pcomp': 45,
 'ADV_amod': 38,
 'PROPN_amod': 242,
 'NOUN_dative': 92,
 'AUX_aux': 390,
 'PROPN_appos': 98,
 'NUM_dep': 7,
 'ADJ_acomp': 56,
 'NUM_compound': 172,
 'VERB_nmod': 54,
 'PROPN_acomp': 1,
 'SCONJ_mark': 72,
 'PRON_dobj': 70,
 'AUX_auxpass': 49,
 'ADJ_ROOT': 29,
 'NOUN_amod': 58,
 'PROPN_prep': 12,

In [20]:
from collections import Counter

common_pos_dep = Counter(count_dependency_dict).most_common(40)
important_pos_dep = []

for pos_dep in common_pos_dep:
    important_pos_dep.append(pos_dep[0])
    
print(important_pos_dep)

['NOUN_compound', 'ADJ_amod', 'NOUN_dobj', 'PROPN_compound', 'NOUN_nsubj', 'VERB_ROOT', 'VERB_ccomp', 'NUM_nummod', 'ADV_advmod', 'PROPN_nsubj', 'VERB_acl', 'NOUN_nmod', 'NOUN_npadvmod', 'VERB_amod', 'VERB_advcl', 'PROPN_nmod', 'VERB_conj', 'VERB_xcomp', 'NOUN_pobj', 'ADP_prep', 'AUX_aux', 'PROPN_dobj', 'NOUN_appos', 'PROPN_amod', 'VERB_compound', 'NUM_compound', 'PROPN_npadvmod', 'VERB_prep', 'VERB_relcl', 'VERB_nsubj', 'VERB_dobj', 'NOUN_ROOT', 'PRON_nsubj', 'PROPN_appos', 'ADJ_advmod', 'ADJ_compound', 'NOUN_dative', 'VERB_dep', 'VERB_csubj', 'PROPN_pobj']


In [21]:
def get_dep_vector(dep_sentence): 
    pos_tags = {'ADJ':0,'ADV':0,'NOUN':0,'PROPN':0,'NUM':0,} 
    pos_sentence = pos_tag_extraction(dep_sentence).split(" ")
    dep_sentence = dep_feature_extraction(dep_sentence).split(' ')

    sentence_dep_dict = dict.fromkeys(important_pos_dep,0)
    for pos, dep in zip(pos_sentence, dep_sentence):
        print(dep)
        if (pos+"_"+dep) in important_pos_dep and pos in pos_tags:
            if (pos+"_"+dep) in list(sentence_dep_dict.keys()):
                sentence_dep_dict[(pos+"_"+dep)] += 1
    return list(sentence_dep_dict.values())

In [22]:
print(get_dep_vector(df['preprocessed_text'][0]))

compound
compound
nsubj
ROOT
dobj
advmod
npadvmod
amod
npadvmod
nsubj
relcl
amod
compound
nsubj
ccomp
amod
amod
amod
dobj
acl
nsubj
compound
compound
dobj
[3, 4, 3, 2, 2, 0, 0, 0, 1, 1, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [23]:
def pipeline(df):
    final_feature = []
    print(df.shape)
    for row in df['preprocessed_text']:
        temp = []
        temp.extend(get_pos_vector(row))
        temp.extend(get_dep_vector(row))
        final_feature.append(temp)
    df_features = pd.DataFrame(final_feature)
    return df_features

In [24]:
df_preprocessed.head()

,tweet,label,id,preprocessed_text
0,"India's gift of 100,000 COVID-19 vaccines arri...",0,0,india gift vaccines arrived barbados earlier t...
1,As part of the ongoing nationwide vaccination ...,0,1,part ongoing nationwide vaccination rollout se...
2,"Pleased to receive 50,000 doses of Covid-19 va...",0,2,pleased receive doses vaccines india morning g...
3,Four former presidents have banded together fo...,0,3,four former presidents banded together two nat...
4,WSJ: All three of Russia's main intelligence s...,1,4,wsj three russia main intelligence services sr...


In [25]:
final_df = pipeline(df_preprocessed)
final_df

(2122, 4)
compound
compound
nsubj
ROOT
dobj
advmod
npadvmod
amod
npadvmod
nsubj
relcl
amod
compound
nsubj
ccomp
amod
amod
amod
dobj
acl
nsubj
compound
compound
dobj
dep
amod
amod
compound
ROOT
acl
nsubj
amod
compound
nmod
compound
compound
compound
appos
ccomp
dobj
nmod
amod
compound
dobj
amod
compound
nsubj
appos
compound
compound
compound
nmod
amod
compound
compound
compound
nsubj
amod
appos
ROOT
compound
dobj
nummod
amod
nsubj
ROOT
advmod
nummod
amod
compound
dobj
acl
npadvmod
amod
compound
compound
nsubj
acl
xcomp
amod
compound
dobj
nsubj
nummod
nmod
amod
compound
compound
compound
compound
nsubj
ROOT
amod
compound
dobj
amod
amod
compound
nsubj
conj
compound
dobj
amod
nsubj
conj
amod
compound
dobj
prep
pobj
compound
nsubj
compound
dobj
prep
pobj
nummod
amod
nsubj
advmod
conj
xcomp
compound
dobj
ROOT
dobj
compound
compound
nsubj
advmod
amod
nsubj
ccomp
nsubj
acl
xcomp
nsubj
ccomp
dobj
prep
pcomp
advmod
amod
amod
nsubj
ROOT
nsubj
advmod
compound
compound
compound
compound
nsubj
ccomp

npadvmod
compound
compound
compound
compound
nsubj
ccomp
nsubj
ccomp
amod
compound
compound
compound
nsubj
ROOT
advmod
compound
nummod
compound
compound
compound
dobj
nsubj
ROOT
compound
dobj
amod
dobj
advmod
amod
compound
compound
nsubj
relcl
compound
compound
compound
nsubj
ccomp
compound
ccomp
dobj
ccomp
ROOT
compound
compound
compound
dobj
compound
compound
nsubj
ccomp
compound
compound
compound
nsubj
advcl
nummod
compound
dobj
compound
nsubj
ccomp
amod
amod
compound
dobj
advmod
nummod
compound
nmod
amod
compound
compound
nsubj
ROOT
compound
compound
dobj
aux
ROOT
compound
compound
compound
compound
nsubj
dobj
advcl
compound
nmod
amod
compound
compound
nsubj
ccomp
amod
amod
compound
compound
compound
dobj
npadvmod
ROOT
compound
compound
compound
nsubj
acl
compound
dobj
xcomp
compound
compound
compound
nsubj
ccomp
amod
compound
compound
compound
compound
dobj
acl
amod
compound
compound
dobj
nmod
pobj
compound
nsubj
nsubj
ROOT
amod
amod
dobj
compound
compound
nsubj
advcl
dobj
xcomp
d

nsubj
ROOT
amod
compound
nsubj
ccomp
amod
dobj
acl
nmod
compound
compound
compound
compound
compound
nsubj
acl
amod
dobj
ccomp
nmod
amod
dobj
amod
dobj
nmod
amod
compound
nsubj
ROOT
nummod
dobj
advcl
compound
dobj
amod
compound
compound
nsubj
acl
advmod
compound
compound
compound
nmod
amod
dobj
nsubj
nmod
amod
amod
nsubj
ROOT
advmod
npadvmod
amod
compound
nsubj
amod
compound
dobj
compound
nsubj
advmod
amod
ROOT
amod
dobj
prep
amod
compound
pobj
nsubj
ROOT
amod
dobj
nummod
amod
compound
compound
nmod
amod
nsubj
nummod
compound
compound
compound
nsubj
advmod
relcl
dobj
appos
ccomp
compound
compound
dobj
acl
amod
dobj
compound
dobj
nsubj
ROOT
nsubj
ccomp
dobj
ccomp
amod
nmod
amod
dobj
dobj
ccomp
amod
nsubj
ccomp
amod
nsubj
ccomp
amod
compound
dobj
compound
compound
npadvmod
amod
nmod
compound
nummod
compound
nsubj
ROOT
nsubj
npadvmod
ccomp
compound
nsubj
ccomp
amod
compound
dobj
acl
npadvmod
nsubj
ccomp
compound
nummod
dobj
acl
compound
dobj
amod
nsubj
compound
nsubj
ROOT
amod
nsubj
ccomp

nsubj
ROOT
advcl
amod
compound
compound
nsubj
advcl
amod
compound
nsubj
ccomp
amod
nmod
amod
dobj
nummod
npadvmod
compound
nummod
dobj
acl
amod
dobj
dobj
ROOT
compound
dobj
acl
prt
quantmod
quantmod
nummod
compound
compound
compound
compound
compound
dobj
nummod
npadvmod
advcl
amod
compound
dobj
acl
compound
compound
compound
dobj
nummod
nsubj
aux
ROOT
nummod
dobj
nummod
dobj
npadvmod
compound
compound
compound
advcl
nsubj
relcl
dative
compound
dobj
acl
prep
pobj
amod
npadvmod
amod
compound
compound
nsubj
ROOT
dobj
compound
appos
amod
compound
compound
compound
dobj
compound
compound
nsubj
ROOT
compound
dobj
advcl
dobj
xcomp
conj
xcomp
compound
dobj
advcl
advmod
amod
npadvmod
compound
compound
compound
compound
compound
compound
nsubj
compound
compound
compound
compound
compound
nsubj
ROOT
npadvmod
npadvmod
compound
nsubj
ccomp
preconj
compound
amod
compound
compound
dobj
nsubj
advmod
ROOT
nsubj
ccomp
dative
det
compound
dobj
amod
compound
compound
appos
compound
appos
compound
ROOT
co

compound
compound
nsubj
advmod
ROOT
compound
dative
amod
amod
compound
appos
advmod
nummod
compound
nsubj
advcl
dobj
advmod
dobj
prep
nummod
compound
compound
compound
pobj
amod
compound
nsubj
amod
compound
compound
appos
nummod
nsubj
ROOT
npadvmod
amod
nsubjpass
aux
auxpass
ccomp
nummod
npadvmod
conj
nsubj
advmod
ccomp
amod
dobj
nsubj
amod
nsubj
relcl
compound
compound
nsubj
ROOT
advmod
nummod
dobj
prep
amod
nmod
amod
compound
compound
pobj
compound
nsubj
ROOT
amod
compound
nsubj
ccomp
compound
compound
compound
nsubj
compound
dobj
compound
compound
compound
nsubj
aux
ROOT
compound
compound
dobj
npadvmod
advmod
mark
nsubj
appos
advcl
amod
compound
compound
attr
csubj
compound
dobj
ROOT
dobj
acl
amod
compound
compound
dobj
dobj
nummod
dobj
nummod
npadvmod
amod
amod
nsubj
ccomp
compound
dobj
ccomp
compound
dobj
conj
ccomp
npadvmod
compound
compound
compound
npadvmod
compound
nummod
amod
advmod
compound
compound
compound
nsubj
ROOT
amod
amod
nsubj
ccomp
compound
nsubj
ccomp
compound
dobj

...

prep
pobj
aux
ROOT
xcomp
dative
compound
dobj
aux
acl
amod
dobj
intj
conj
advmod
oprd
aux
xcomp
amod
dobj
aux
conj
dobj
punct
compound
compound
dobj
compound
compound
compound
compound
nsubj
ROOT
compound
nsubj
ccomp
amod
dobj
ccomp
nsubj
ccomp
compound
compound
dobj
advcl
dobj
compound
compound
compound
nsubj
advmod
appos
ROOT
compound
compound
nsubj
ccomp
amod
compound
compound
dobj
amod
nsubj
ccomp
nummod
npadvmod
compound
compound
npadvmod
nummod
nmod
amod
nsubj
relcl
nsubj
aux
nsubj
aux
ROOT
amod
compound
compound
dobj
advcl
nsubj
advmod
ccomp
compound
compound
compound
compound
dobj
compound
npadvmod
csubj
amod
dobj
acl
dobj
ROOT
nummod
nsubj
aux
ccomp
dobj
auxpass
ccomp
advmod
amod
amod
dobj
amod
amod
nsubj
ROOT
compound
nsubj
acl
oprd
compound
compound
compound
nsubj
conj
ccomp
compound
compound
dobj
compound
nmod
amod
nmod
amod
amod
compound
compound
npadvmod
nsubj
acl
nmod
amod
dobj
nmod
amod
amod
advmod
ROOT
compound
dobj
amod
amod
compound
compound
nsubj
acl
compound
dobj
a

compound
nsubj
ROOT
compound
nsubj
advmod
ccomp
dobj
compound
compound
dobj
amod
npadvmod
compound
compound
compound
dobj
aux
compound
nmod
acl
compound
compound
nmod
advmod
amod
nsubj
ROOT
compound
nmod
amod
nsubj
prep
nmod
nmod
amod
pobj
ccomp
amod
nsubj
advmod
oprd
nummod
amod
ROOT
compound
compound
compound
compound
appos
nsubj
acl
nsubj
nsubj
ccomp
advmod
intj
ROOT
dobj
prep
pobj
amod
compound
nsubj
amod
compound
nsubj
ccomp
nmod
amod
attr
npadvmod
nummod
nsubj
ROOT
xcomp
amod
compound
dobj
nsubj
ROOT
dobj
amod
compound
dobj
compound
compound
nsubj
nummod
nummod
npadvmod
prep
compound
compound
pobj
cc
pobj
ROOT
compound
nummod
npadvmod
nsubj
relcl
amod
dobj
advmod
ccomp
xcomp
amod
compound
compound
dobj
compound
nmod
amod
compound
compound
nsubj
advmod
ROOT
amod
advmod
dobj
compound
nsubj
acl
compound
dobj
ROOT
compound
nmod
amod
compound
dobj
amod
compound
compound
dobj
ROOT
amod
dobj
advmod
dobj
dep
compound
compound
nsubj
ccomp
amod
compound
compound
dobj
ROOT
compound
compound

nsubj
ccomp
dobj
advmod
nsubj
advcl
advmod
amod
amod
dobj
aux
compound
nsubj
advmod
ROOT
compound
compound
compound
dobj
det
compound
nsubj
ccomp
compound
dobj
dep
ROOT
dobj
nmod
compound
amod
npadvmod
amod
compound
compound
compound
compound
compound
compound
compound
nmod
compound
compound
compound
compound
dobj
compound
advmod
compound
compound
compound
nsubj
prep
nummod
compound
compound
compound
pobj
advmod
ROOT
dep
acl
compound
amod
nmod
amod
compound
compound
compound
dobj
nummod
amod
nsubj
ROOT
compound
dobj
intj
amod
nsubj
xcomp
compound
nsubj
ccomp
xcomp
nsubj
compound
compound
nsubj
ROOT
nmod
amod
dobj
acl
dobj
conj
npadvmod
nsubj
aux
ROOT
dobj
prep
pcomp
amod
compound
compound
dobj
advmod
compound
dobj
acl
compound
dobj
acl
dobj
advcl
amod
compound
compound
dobj
nsubj
advmod
ROOT
compound
advmod
compound
ccomp
compound
compound
nsubj
acl
compound
dobj
ccomp
advmod
dobj
amod
nsubj
advcl
amod
amod
dobj
compound
nsubj
ROOT
compound
dobj
amod
nsubj
ccomp
dobj
nsubj
ccomp
ccomp


compound
nsubj
acl
ROOT
compound
compound
compound
nsubj
ccomp
nsubj
ccomp
dobj
acl
nsubj
aux
ccomp
compound
dobj
ROOT
dobj
npadvmod
dep
amod
amod
compound
compound
nmod
amod
amod
amod
compound
dobj
acl
xcomp
nmod
amod
dobj
nummod
nsubj
conj
nummod
dobj
dobj
dobj
nummod
amod
dobj
compound
compound
compound
compound
compound
nsubj
aux
advmod
ROOT
amod
dobj
compound
npadvmod
prep
pobj
compound
compound
nsubj
prep
npadvmod
ROOT
amod
amod
dobj
ccomp
nsubj
ccomp
advmod
dobj
dep
amod
compound
compound
nsubj
aux
compound
amod
compound
compound
dobj
amod
compound
compound
compound
compound
compound
nsubj
ROOT
compound
dobj
npadvmod
nummod
dobj
acl
npadvmod
amod
dobj
advmod
amod
dobj
compound
nsubj
advmod
ROOT
compound
nummod
amod
nmod
compound
compound
dobj
acl
compound
dobj
advcl
xcomp
amod
aux
ccomp
advmod
advmod
advmod
dobj
amod
nsubj
ROOT
xcomp
compound
nsubj
ccomp
amod
amod
nmod
amod
advmod
amod
compound
dobj
acl
advmod
dep
nsubj
aux
neg
ccomp
advmod
compound
dobj
compound
nmod
acl
compou

,0,1,2,3,4,5,6,7,8,9,...,38,39,40,41,42,43,44,45,46,47
0,0,0,0,0,0,0,0,0,3,4,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,2,4,...,0,1,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,3,3,...,0,1,0,0,0,1,0,0,0,0
3,0,0,0,0,0,0,0,0,4,2,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,4,6,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2117,0,0,0,0,0,0,0,0,8,1,...,0,0,0,0,0,0,0,0,0,0
2118,0,0,0,0,0,0,0,0,3,2,...,0,0,0,0,1,0,0,0,0,0
2119,0,0,0,0,0,0,0,0,2,1,...,0,0,0,0,0,0,0,0,0,0
2120,0,0,0,0,0,0,0,0,5,5,...,0,1,0,0,0,0,0,0,0,0


In [27]:
X = final_df.copy()
y = df_preprocessed.label

In [28]:
X_train, X_test, y_train, y_test = train_test_split(X, y,train_size=0.8,random_state=40,stratify=y)
# X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, train_size=0.70, random_state=42)

In [29]:
X_train

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47
1069,0,0,0,0,0,0,0,0,1,2,1,1,1,0,0,2,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
1585,0,0,0,0,0,0,0,0,4,2,1,3,3,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
863,0,0,0,0,0,0,0,0,5,5,3,1,1,0,0,3,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
844,0,0,0,0,0,0,0,0,8,2,2,1,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
531,0,0,0,0,0,0,0,0,5,3,3,4,1,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
226,0,0,0,0,0,0,0,0,2,1,2,0,3,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
720,0,0,0,0,0,0,0,0,3,3,2,5,3,0,0,1,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1882,0,0,0,0,0,0,0,0,2,3,1,1,4,0,0,0,2,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
214,0,0,0,0,0,0,0,0,2,1,3,0,2,0,0,1,4,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [51]:
model = SentenceTransformer('vinai/bertweet-covid19-base-uncased')


No sentence-transformers model found with name /home/ayush/.cache/torch/sentence_transformers/vinai_bertweet-covid19-base-uncased. Creating a new one with MEAN pooling.
Some weights of the model checkpoint at /home/ayush/.cache/torch/sentence_transformers/vinai_bertweet-covid19-base-uncased were not used when initializing RobertaModel: ['lm_head.decoder.bias', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.decoder.weight', 'lm_head.bias', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
emoji is not installed, thus not c

IndexError: index out of range in self

In [ ]:
embeddings = []
for i in range(len(final_df)):
    embeddings.append(model.encode(df_preprocessed['preprocessed_text'][i],batch_size= 32,convert_to_numpy=True ))

In [ ]:
type(embeddings)

In [34]:
model = SVC()
model.fit(X_train, y_train)

SVC()

In [41]:
model.score(X_test, y_test)

0.788235294117647

In [42]:
y_pred = model.predict(X_test)

In [43]:
print(accuracy_score(y_test, y_pred))

0.788235294117647


In [44]:
model2 = LogisticRegression()
model2.fit(X_train, y_train)

LogisticRegression()

In [47]:
y_pred2 = model2.predict(X_test)

In [48]:
accuracy_score(y_test, y_pred2)

0.788235294117647

In [ ]:
model3 = SentenceTransformer('distilbert-base-nli-mean-tokens')
embeddings = []
for i in range(len(final_df)):
    embeddings.append(model.encode(final_df['preprocessed_text'][i]))

In [ ]:
# embeddings